In [1]:
import pandas as pd

In [3]:
# Article dataset to ingest into our knowledge base
DATA_FP = "../cnn_dailymail/3.0.0/train-00000-of-00003.parquet"
NUM_ARTICLES = 100  # We input 100 articles only for testing purposes

# We use Milvus vector database for knowledge base
MILVUS_URL = "http://localhost:19530"
MILVUS_HOST = MILVUS_URL.split("//")[-1].split(":")[0]
MILVUS_PORT = MILVUS_URL.split(":")[-1]
MILVUS_COLLECTION_NAME = "articles_collection"  # This is like a table in a database
VECTOR_DIM = 1024  # Dimension of the vectors we store in Milvus
EMBEDDING_MODEL_NAME = "mixedbread-ai/mxbai-embed-large-v1"

# vLLM OpenAI Compatible Server for LLM
VLLM_HOST = "http://localhost:8000/v1"
LLM_NAME = "astronomer/Llama-3-8B-Instruct-GPTQ-4-Bit"  # We use Llama3-8B with GPTQ 4-bit quantization
MAX_TOKENS = 1024  # Maximum number of output tokens
TEMPERATURE = 0  # This parameter controls the creativity of the output

In [4]:
# Read the input data
df = pd.read_parquet(DATA_FP)
df = df[["id", "article"]]
# Investigate an article
print(df.iloc[0]["article"])
# Get all articles in a list format
articles = df.iloc[:NUM_ARTICLES].to_dict("records")

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

# Ingest data to Milvus

In [5]:
# Ingest data into Milvus (or any vector database of your choice)
from pymilvus import (
    MilvusClient,
    DataType,
)
from sentence_transformers import SentenceTransformer

# Connect to Milvus
milvus_client = MilvusClient(uri=MILVUS_HOST)

# Create a collection to store vectors and metadata,
# think of it as a table in a traditional SQL database.
schema = milvus_client.create_schema(auto_id=False, enable_dynamic_field=False)
schema.add_field(
    schema=schema,
    field_name="id",
    datatype=DataType.VARCHAR,
    is_primary=True,
    max_length=10000,
)
# This is the vector embedding
schema.add_field(
    schema=schema, field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=VECTOR_DIM
)
# This is the corresponding raw text for the vector
schema.add_field(
    schema=schema, field_name="text", datatype=DataType.VARCHAR, max_length=10000
)

# Drop the old collection and create a new one
if milvus_client.has_collection(MILVUS_COLLECTION_NAME):
    milvus_client.drop_collection(MILVUS_COLLECTION_NAME)

milvus_client.create_collection(collection_name=MILVUS_COLLECTION_NAME, schema=schema)

# Define an embedding model to create vectors from texts,
# you can replace by other models
transformer = SentenceTransformer(EMBEDDING_MODEL_NAME)

# Loop over articles and create embedding vectors
data = []
for article in articles:
    data.append(
        {
            "id": article["id"],
            "vector": transformer.encode(article["article"]),
            "text": article["article"],
        }
    )
res = milvus_client.insert(collection_name=MILVUS_COLLECTION_NAME, data=data)

/home/quandv/anaconda3/envs/sup/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/quandv/anaconda3/envs/sup/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Query data via Langchain

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms.vllm import VLLMOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores.milvus import Milvus

# We access vLLM via OpenAI compatible server
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=VLLM_HOST,
    model_name=LLM_NAME,
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    streaming=True,
    verbose=False,
)

# We our embedding model reside on cpu
model_kwargs = {"trust_remote_code": True, "device": "cpu"}

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME, model_kwargs=model_kwargs, show_progress=False
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
    collection_name=MILVUS_COLLECTION_NAME,
    text_field="text",
    drop_old=False,
)
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

/home/quandv/anaconda3/envs/sup/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/quandv/anaconda3/envs/sup/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 0.3.0. An updated version of the class exists in the langchain-milvus package and should be used instead. To use it run `pip install -U langchain-milvus` and import as `from langchain_milvus import MilvusVectorStore`.
  warn_deprecated(


In [10]:
# Test our chain (retriever and LLM) with a question
question = "Tell me some news about Harry Potter?"
result = qa_chain.invoke({"query": question})
print(result)

{'query': 'Tell me some news about Harry Potter?', 'result': " According to the article, Daniel Radcliffe, the actor who plays Harry Potter, has turned 18 and will be able to access a reported £20 million ($41.1 million) fortune. However, he has stated that he doesn't plan to spend his money on lavish things like fast cars and celebrity parties. Instead, he likes to buy simple things like books, CDs, and DVDs. He also plans to continue acting in films and has already started working on a new project.", 'source_documents': [Document(page_content='LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parti